In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime, timedelta
np.random.seed(1)

In [2]:
ks_meta = pd.read_csv('ks_meta_feature_dataset.csv')

In [136]:
ks_meta.columns

Index(['category_name', 'location_name', 'country_displayable_name',
       'usd_goal', 'set_fundraising_period', 'target_goal_period_rate',
       'date_launched_year', 'date_launched_month', 'launching_delay_time',
       'disable_communication', 'is_starrable', 'state'],
      dtype='object')

In [4]:
ks_df = pd.read_csv('ks_overall_dataset.csv')

In [139]:
ks_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200232 entries, 0 to 200231
Data columns (total 27 columns):
blurb                       200230 non-null object
category_name               200232 non-null object
category_specific           200232 non-null object
country_displayable_name    200232 non-null object
created_at                  200232 non-null int64
date_created                200232 non-null object
deadline                    200232 non-null int64
date_deadline               200232 non-null object
disable_communication       200232 non-null bool
id                          200232 non-null int64
is_starrable                200232 non-null bool
launched_at                 200232 non-null int64
date_launched               200232 non-null object
date_launched_year          200232 non-null int64
date_launched_month         200232 non-null int64
location_name               200025 non-null object
name                        200232 non-null object
photo_link                  2002

In [6]:
ks_2019 = ks_df[ks_df['date_launched_year'] == 2019]

In [7]:
ks_2019['state'].value_counts()

successful    30002
failed         7201
Name: state, dtype: int64

In [8]:
ks_meta_2019 = ks_2019[['category_name', 'location_name', 'country_displayable_name',
       'usd_goal', 'set_fundraising_period', 'target_goal_period_rate',
       'date_launched_month', 'launching_delay_time', 'disable_communication',
       'is_starrable', 'state']]

In [9]:
ks_meta_2019.columns

Index(['category_name', 'location_name', 'country_displayable_name',
       'usd_goal', 'set_fundraising_period', 'target_goal_period_rate',
       'date_launched_month', 'launching_delay_time', 'disable_communication',
       'is_starrable', 'state'],
      dtype='object')

In [10]:
ks_meta_2019.head(1)

category_name location_name country_displayable_name    usd_goal  \
0  Children's Books   Ste.-Maxime                   France  684.396336   

   set_fundraising_period  target_goal_period_rate  date_launched_month  \
0                 5184000                 0.000132                    6   

   launching_delay_time  disable_communication  is_starrable       state  
0                  5548                  False         False  successful

In [11]:
ks_meta_2019.isna().sum()

category_name               0
location_name               0
country_displayable_name    0
usd_goal                    0
set_fundraising_period      0
target_goal_period_rate     0
date_launched_month         0
launching_delay_time        0
disable_communication       0
is_starrable                0
state                       0
dtype: int64

In [12]:
print(ks_meta_2019['state'].value_counts())
ks_meta_2019['state'].value_counts(normalize=True)

successful    30002
failed         7201
Name: state, dtype: int64


successful    0.80644
failed        0.19356
Name: state, dtype: float64

In [14]:
#IQR 기반 이상치 검출
#IQR : 3분위수-1분위수
#이상치 기준 (rate는 일반적으로 1.5사용)
#1분위수 - IQR*rate 보다 작은수
#3분위 + IQR*rate 보다 큰수

def get_outlier(data, rate=1.5):
    q1 = np.quantile(data, q=0.25)
    q3 = np.quantile(date, q=0.75)
    IQR = q3 - q1
    return (data < q1 - IQR * rate) | (data > q3 + IQR * rate)

In [25]:
column_to_look = ks_meta_2019['usd_goal']

print("max:",column_to_look.max())
print("min:",column_to_look.min())
print("mean:",column_to_look.mean())
print("median:",column_to_look.median())
print("mode:",column_to_look.mode())
print("std:",column_to_look.std())
print("25%:",column_to_look.quantile(0.25))
print("50%:",column_to_look.quantile(0.5))
print("75%:",column_to_look.quantile(0.75))
print("null/nan 수: ", column_to_look.isna().sum())

max: 100000000.0
min: 0.5114279
mean: 17009.7170924536
median: 3684.2997299999997
mode: 0    5000.0
dtype: float64
std: 532309.2513937358
25%: 1000.0
50%: 3684.2997299999997
75%: 10000.0
null/nan 수:  0


In [138]:
ks_meta_2019['usd_goal'].sort_values(ascending=False).head() #이건 scaling 후에 다시 확인한 것

15985    187.831291
32265     18.754370
27847     18.754370
29915     18.595053
37003     10.827192
Name: usd_goal, dtype: float64

In [20]:
ks_meta_2019['usd_goal'].value_counts().sort_values(ascending=False).head()

5000.0     1655
10000.0    1642
500.0      1086
1000.0     1082
2000.0      897
Name: usd_goal, dtype: int64

In [26]:
column_to_look = ks_meta_2019['set_fundraising_period']

print("max:",column_to_look.max())
print("min:",column_to_look.min())
print("mean:",column_to_look.mean())
print("median:",column_to_look.median())
print("mode:",column_to_look.mode())
print("std:",column_to_look.std())
print("25%:",column_to_look.quantile(0.25))
print("50%:",column_to_look.quantile(0.5))
print("75%:",column_to_look.quantile(0.75))
print("null/nan 수: ", column_to_look.isna().sum())

max: 8448346
min: 86400
mean: 2754649.5720775207
median: 2592000.0
mode: 0    2592000
dtype: int64
std: 1038867.5425734699
25%: 2419248.0
50%: 2592000.0
75%: 2869922.5
null/nan 수:  0


In [71]:
column_to_look = ks_meta_2019['target_goal_period_rate']

print("max:",column_to_look.max())
print("min:",column_to_look.min())
print("mean:",column_to_look.mean())
print("median:",column_to_look.median())
print("mode:",column_to_look.mode())
print("std:",column_to_look.std())
print("25%:",column_to_look.quantile(0.25))
print("50%:",column_to_look.quantile(0.5))
print("75%:",column_to_look.quantile(0.75))
print("null/nan 수: ", column_to_look.isna().sum())

max: 19.276736833988746
min: 1.2406508597424632e-07
mean: 0.005502005278324271
median: 0.0013770570966599466
mode: 0    0.001929
dtype: float64
std: 0.10446809724034069
25%: 0.0003906585238054575
50%: 0.0013770570966599466
75%: 0.003858024691358025
null/nan 수:  0


In [33]:
ks_meta_2019.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37203 entries, 0 to 200229
Data columns (total 11 columns):
category_name               37203 non-null object
location_name               37203 non-null object
country_displayable_name    37203 non-null object
usd_goal                    37203 non-null float64
set_fundraising_period      37203 non-null int64
target_goal_period_rate     37203 non-null float64
date_launched_month         37203 non-null object
launching_delay_time        37203 non-null int64
disable_communication       37203 non-null bool
is_starrable                37203 non-null bool
state                       37203 non-null object
dtypes: bool(2), float64(2), int64(2), object(5)
memory usage: 2.9+ MB


In [35]:
ks_meta_2019['date_launched_month'] = ks_meta_2019['date_launched_month'].astype('str')

/opt/anaconda3/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [93]:
ks_meta_2019 = ks_meta_2019.reset_index(drop=True)

In [ ]:
#label encoding : disable_communication, is_starrable, state
#one hot encoding : category_name, location_name, country_displayable_name, date_launched_month
#standard scaling : usd_goal, set_fundraising_period, target_goal_period_rate, launching_delay_time

In [40]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

In [55]:
col_to_label_encode = ['disable_communication', 'is_starrable', 'state']
col_to_oh_encode = ['category_name', 'location_name', 'country_displayable_name', 'date_launched_month']
col_to_scale = ['usd_goal', 'set_fundraising_period', 'target_goal_period_rate', 'launching_delay_time']

In [56]:
#Label Encoding

def l_encoding(x):
    le = LabelEncoder()
    le.fit(x)
    l_encoded_x = le.transform(x)
    return l_encoded_x

In [51]:
ks_meta_2019['state'].head()

0     successful
7         failed
8     successful
12    successful
18        failed
Name: state, dtype: object

In [118]:
col_to_label_encode = ['disable_communication', 'is_starrable', 'state']

ks_meta_2019[col_to_label_encode] = ks_meta_2019[col_to_label_encode].apply(l_encoding)
ks_meta_2019[col_to_label_encode]

disable_communication  is_starrable  state
0                          0             0      1
1                          0             0      0
2                          0             0      1
3                          0             0      1
4                          0             0      0
...                      ...           ...    ...
37198                      0             0      1
37199                      0             0      1
37200                      0             0      0
37201                      0             0      0
37202                      0             0      0

[37203 rows x 3 columns]

In [121]:
ks_meta_2019.head(1)

category_name location_name country_displayable_name    usd_goal  \
0  Children's Books   Ste.-Maxime                   France  684.396336   

   set_fundraising_period  target_goal_period_rate date_launched_month  \
0                 5184000                 0.000132                   6   

   launching_delay_time  disable_communication  is_starrable  state  
0                  5548                      0             0      1

In [122]:
ks_meta_2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37203 entries, 0 to 37202
Data columns (total 11 columns):
category_name               37203 non-null object
location_name               37203 non-null object
country_displayable_name    37203 non-null object
usd_goal                    37203 non-null float64
set_fundraising_period      37203 non-null int64
target_goal_period_rate     37203 non-null float64
date_launched_month         37203 non-null object
launching_delay_time        37203 non-null int64
disable_communication       37203 non-null int64
is_starrable                37203 non-null int64
state                       37203 non-null int64
dtypes: float64(2), int64(5), object(4)
memory usage: 3.1+ MB


In [125]:
#One-Hot Encoding -- Pandas.get_dummies()

col_to_oh_encode = ['category_name', 'location_name', 'country_displayable_name', 'date_launched_month']

ks_meta_2019 = pd.get_dummies(ks_meta_2019, columns=col_to_oh_encode)
ks_meta_2019

usd_goal  set_fundraising_period  target_goal_period_rate  \
0         684.396336                 5184000                 0.000132   
1       30000.000000                 2592000                 0.011574   
2        1009.040070                 2156400                 0.000468   
3        7000.000000                 2595600                 0.002697   
4      300000.000000                 3888000                 0.077160   
...              ...                     ...                      ...   
37198     393.166059                 2592000                 0.000152   
37199    1305.500070                 2592000                 0.000504   
37200    7893.879210                 1728000                 0.004568   
37201   20000.000000                 5011887                 0.003991   
37202     700.000000                 2657079                 0.000263   

       launching_delay_time  disable_communication  is_starrable  state  \
0                      5548                      0             0      1   
1                  12384398                      0             0      0   
2                   3564810                      0             0      1   
3                    153831                      0             0      1   
4                    680338                      0             0      0   
...                     ...                    ...           ...    ...   
37198                655598                      0             0      1   
37199                171726                      0             0      1   
37200                245102                      0             0      0   
37201               2599879                      0             0      0   
37202               2568846                      0             0      0   

       category_name_3D Printing  category_name_Academic  \
0                              0                       0   
1                              0                       0   
2                              0                       0   
3                              0                       0   
4                              0                       0   
...                          ...                     ...   
37198                          0                       0   
37199                          0                       0   
37200                          0                       0   
37201                          0                       0   
37202                          0                       0   

       category_name_Accessories  ...  date_launched_month_11  \
0                              0  ...                       0   
1                              0  ...                       0   
2                              0  ...                       0   
3                              0  ...                       0   
4                              0  ...                       0   
...                          ...  ...                     ...   
37198                          1  ...                       0   
37199                          1  ...                       0   
37200                          0  ...                       0   
37201                          0  ...                       0   
37202                          0  ...                       0   

       date_launched_month_12  date_launched_month_2  date_launched_month_3  \
0                           0                      0                      0   
1                           0                      0                      0   
2                           0                      1                      0   
3                           0                      0                      0   
4                           0                      0                      0   
...                       ...                    ...                    ...   
37198                       0                      0                      1   
37199                       0                      0                      0   
37200                

In [126]:
ks_meta_2019.shape

(37203, 3652)

In [127]:
#Standard Scaling

col_to_scale = ['usd_goal', 'set_fundraising_period', 'target_goal_period_rate', 'launching_delay_time']

s_scaler = StandardScaler()
ks_meta_2019[col_to_scale] = s_scaler.fit_transform(ks_meta_2019[col_to_scale])

In [128]:
ks_meta_2019[col_to_scale]

usd_goal  set_fundraising_period  target_goal_period_rate  \
0     -0.030669                2.338492                -0.051404   
1      0.024404               -0.156566                 0.058124   
2     -0.030059               -0.575875                -0.048188   
3     -0.018805               -0.153101                -0.026852   
4      0.531635                1.090963                 0.685946   
...         ...                     ...                      ...   
37198 -0.031216               -0.156566                -0.051216   
37199 -0.029502               -0.156566                -0.047846   
37200 -0.017125               -0.988252                -0.008939   
37201  0.005618                2.172816                -0.014469   
37202 -0.030640               -0.093921                -0.050146   

       launching_delay_time  
0                 -0.348648  
1                  0.467476  
2                 -0.113990  
3                 -0.338872  
4                 -0.304160  
...                     ...  
37198             -0.305791  
37199             -0.337692  
37200             -0.332855  
37201             -0.177607  
37202             -0.179653  

[37203 rows x 4 columns]

In [131]:
print(ks_meta_2019.shape)
ks_meta_2019.head()

(37203, 3652)


usd_goal  set_fundraising_period  target_goal_period_rate  \
0 -0.030669                2.338492                -0.051404   
1  0.024404               -0.156566                 0.058124   
2 -0.030059               -0.575875                -0.048188   
3 -0.018805               -0.153101                -0.026852   
4  0.531635                1.090963                 0.685946   

   launching_delay_time  disable_communication  is_starrable  state  \
0             -0.348648                      0             0      1   
1              0.467476                      0             0      0   
2             -0.113990                      0             0      1   
3             -0.338872                      0             0      1   
4             -0.304160                      0             0      0   

   category_name_3D Printing  category_name_Academic  \
0                          0                       0   
1                          0                       0   
2                          0                       0   
3                          0                       0   
4                          0                       0   

   category_name_Accessories  ...  date_launched_month_11  \
0                          0  ...                       0   
1                          0  ...                       0   
2                          0  ...                       0   
3                          0  ...                       0   
4                          0  ...                       0   

   date_launched_month_12  date_launched_month_2  date_launched_month_3  \
0                       0                      0                      0   
1                       0                      0                      0   
2                       0                      1                      0   
3                       0                      0                      0   
4                       0                      0                      0   

   date_launched_month_4  date_launched_month_5  date_launched_month_6  \
0                      0                      0                      1   
1                      0                      0                      0   
2                      0                      0                      0   
3                      0                      0                      0   
4                      0                      0                      0   

   date_launched_month_7  date_launched_month_8  date_launched_month_9  
0                      0                      0                      0  
1                      0                      1                      0  
2                      0                      0                      0  
3                      0                      0                      0  
4                      1                      0                      0  

[5 rows x 3652 columns]

In [137]:
ks_meta_2019.to_csv('ks_metadata_2019_preprocessed.csv', index=False, encoding='utf-8')

In [ ]:
#원래는 train, test 먼저 나누고 하지만 (x_test는 새로 들어오는 데이터의 개념이기 때문에), 난 이미지와 텍스트 데이터까지 다 합쳐서 분류 모델 돌릴 것이기에 일단은 전처리만 해준다

In [ ]:
#이상치 관련 전처리는 데이터 다 합치게 되면 그때 하는 것으로 하자

In [ ]:
#아니면, 여기 위에서 했던 것들 한번에 처리할 수 있는 모듈 하나 만들어서 년도별 데이터 완성 될 때마다 돌려도 되겠다